In [ ]:
import sys
import os
import pandas as pd
sys.path.append('/home/vs428/Documents/DischargeMe/hail-dischargeme/scoring/scoring.py')

In [ ]:
challenge_data_fp = "/gpfs/gibbs/project/rtaylor/shared/DischargeMe/public/"


In [ ]:
challenge_data_fp + "train/simple_train.json"

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset('json', data_files=challenge_data_fp + "train/simple_train.json", split='train')
eval_dataset = load_dataset('json', data_files=challenge_data_fp + "valid/simple_valid.json", split='train')

In [ ]:
eval_dataset = eval_dataset.select(range(500))

In [ ]:
import openai
openai.api_type = "azure"
openai.api_base = "https://decile-openai-llm.openai.azure.com/" #os.getenv("OPENAI_API_BASE")
openai.api_version = "2023-07-01-preview"
openai.api_key = "bfdc4ba059264591a99da53f0fbb71ca"#os.getenv("OPENAI_API_KEY")
engine = "decile-gpt-35-turbo-16k"

# AZURE_OPENAI_ENDPOINT="https://decile-openai-llm.openai.azure.com/"
# OLD_AZURE_OPENAI_KEY="67e338a6726c4c1981d15686d55bd841"
# AZURE_OPENAI_KEY="bfdc4ba059264591a99da53f0fbb71ca"

In [ ]:
gpt_inputs = []
for datum in eval_dataset:
    message_text = [{"role":"system","content":"You are a physician generating a summary brief hospital course from the patient encounter information given"}]
    
    gpt_service_prompt = {"role":"user",
                     "content":f"Summarize the following patient hospital encounter into a brief hospital course:\n\n{datum['input']}" }
    
    message_text.append(gpt_service_prompt)
    
    # print(f"Deprescribe Prompt: {message_text}")
    gpt_inputs.append(message_text)

In [ ]:
len(gpt_inputs)

In [ ]:
completions = []
for idx, gpt_input in enumerate(gpt_inputs[:10]):
    completion = openai.ChatCompletion.create(
      engine=engine,
      messages = gpt_input,
    )
    print(idx)
    completions.append(completion['choices'][0]['message']['content'])

In [ ]:
completions

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', "rouge2", 'rougeL'], use_stemmer=True)


In [ ]:
all_scores = []
for gen, ref in zip(completions, eval_dataset['output'][:10]):
    scores = scorer.score(gen, ref)
    print(scores)
    scores_dict = {"rouge1_precision":scores['rouge1'][0], 
                  "rouge1_recall":scores['rouge1'][1], 
                  "rouge1_f1":scores['rouge1'][2], 
                  "rouge2_precision":scores['rouge2'][0], 
                  "rouge2_recall":scores['rouge2'][1], 
                  "rouge2_f1":scores['rouge2'][2], 
                  "rougeL_precision":scores['rougeL'][0], 
                  "rougeL_recall":scores['rougeL'][1], 
                  "rougeL_f1":scores['rougeL'][2], }
    all_scores.append(scores_dict)

In [ ]:

def add_line_breaks(text):
    return text.replace('\n', '<br>')


In [ ]:
comparison_df = pd.DataFrame([completions, eval_dataset['output'][:10]]).T.rename({0:"GPT", 1:"gold-standard"}, axis=1)

In [ ]:
comparison_df_styled = comparison_df.style.format({'completion_text': add_line_breaks})


In [ ]:

with open(f"comparison_simple_gpt3.5.html", "w") as file:
    file.write(comparison_df_styled.to_html())

In [ ]:
import pandas as pd
pd.DataFrame.from_records(all_scores).describe()

# One-shot

In [ ]:
examples = ["Example\n######\n\n" + ex for ex in eval_dataset['output'][5:7]]

In [ ]:
print(examples[0])

In [ ]:
gpt_inputs = []
for datum in eval_dataset:
    message_text = [{"role":"system","content":"You are a physician generating a summary brief hospital course from the patient encounter information given"}]
    
    gpt_service_prompt = {"role":"user",
                     "content":f"Summarize the following patient hospital encounter into a brief hospital course:\n\n{datum['input']}.\n\nUse these two as examples:\n\n{'\n\n'.join(examples)}" }
    
    message_text.append(gpt_service_prompt)
    
    # print(f"Deprescribe Prompt: {message_text}")
    gpt_inputs.append(message_text)

In [ ]:
len(gpt_inputs)

In [ ]:
print(gpt_inputs[0][1]['content'])

In [ ]:
completions = []
for idx, gpt_input in enumerate(gpt_inputs[:10]):
    completion = openai.ChatCompletion.create(
      engine=engine,
      messages = gpt_input,
    )
    print(idx)
    completions.append(completion['choices'][0]['message']['content'])

In [ ]:
completions

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', "rouge2", 'rougeL'], use_stemmer=True)


In [ ]:
all_scores = []
for gen, ref in zip(completions, eval_dataset['output'][:10]):
    scores = scorer.score(gen, ref)
    print(scores)
    scores_dict = {"rouge1_precision":scores['rouge1'][0], 
                  "rouge1_recall":scores['rouge1'][1], 
                  "rouge1_f1":scores['rouge1'][2], 
                  "rouge2_precision":scores['rouge2'][0], 
                  "rouge2_recall":scores['rouge2'][1], 
                  "rouge2_f1":scores['rouge2'][2], 
                  "rougeL_precision":scores['rougeL'][0], 
                  "rougeL_recall":scores['rougeL'][1], 
                  "rougeL_f1":scores['rougeL'][2], }
    all_scores.append(scores_dict)

In [ ]:

def add_line_breaks(text):
    return text.replace('\n', '<br>')


In [ ]:
comparison_df = pd.DataFrame([completions, eval_dataset['output'][:10]]).T.rename({0:"GPT", 1:"gold-standard"}, axis=1)

In [ ]:
comparison_df_styled = comparison_df.style.format({'completion_text': add_line_breaks})


In [ ]:

with open(f"comparison_simple_gpt3.5_2shot.html", "w") as file:
    file.write(comparison_df_styled.to_html())

In [ ]:
import pandas as pd
pd.DataFrame.from_records(all_scores).describe()